In [1]:
from NLPutils.models import BLSTM

In [5]:
import tensorflow as tf

sess = tf.Session()

model = BLSTM.restore(sess, '/data/models/conll-chunk', 'chunk')

Loaded saver def
Imported graph def
Failed to get transition matrix, setting crf=False


In [6]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import utils
from tensorflow.python.util import compat

In [7]:
export_path = "/data/frozen_test"
builder = saved_model_builder.SavedModelBuilder(export_path)

In [12]:
# Build the signature_def_map.
# classification_inputs = utils.build_tensor_info(serialized_tf_example)
# classification_outputs_classes = utils.build_tensor_info(prediction_classes)
# classification_outputs_scores = utils.build_tensor_info(values)

# classification_signature = signature_def_utils.build_signature_def(
#   inputs={signature_constants.CLASSIFY_INPUTS: classification_inputs},
#   outputs={
#       signature_constants.CLASSIFY_OUTPUT_CLASSES:
#           classification_outputs_classes,
#       signature_constants.CLASSIFY_OUTPUT_SCORES:
#           classification_outputs_scores
#   },
# method_name=signature_constants.CLASSIFY_METHOD_NAME)

tensor_info_x = utils.build_tensor_info(model.x)
tensor_info_xch = utils.build_tensor_info(model.xch)
tensor_info_out = utils.build_tensor_info(model.probs)
tensor_info_transition = utils.build_tensor_info(model.A)

AttributeError: 'NoneType' object has no attribute 'dtype'

In [11]:
prediction_signature = signature_def_utils.build_signature_def(
      inputs={'x': tensor_info_x,
              'xch': tensor_info_xch},
      outputs={'scores': tensor_info_out},
method_name=signature_constants.CLASSIFY_METHOD_NAME)

crf_signature = signature_def_utils.build_signature_def(
      inputs={'x': tensor_info_x,
              'xch': tensor_info_xch},
      outputs={'scores': tensor_info_out,
               'transitions': tensor_info_transition},
method_name=signature_constants.PREDICT_METHOD_NAME)

NameError: name 'tensor_info_transition' is not defined

In [10]:
legacy_init_op = tf.group(tf.initialize_all_tables(), name='legacy_init_op')

builder.add_meta_graph_and_variables(
  sess, [tag_constants.SERVING],
  signature_def_map={
      'predict_ner':
          crf_signature,
      signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
          prediction_signature
  },
  legacy_init_op=legacy_init_op)

builder.save()

Instructions for updating:
Use `tf.tables_initializer` instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'/data/frozen_test/saved_model.pb'


b'/data/frozen_test/saved_model.pb'